In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from parse_excel import Parser
from fuzzywuzzy import process

In [2]:
BASE_DIR = "fixationcount_forframes"
CODE_TYPE = "static"
OUT_DIR = "otuput"
DATABASE_MATRIX = "Database_matrix_modified_anx.xlsx"
PREFIX = "fixationrate_aoi_clubfix_"
PREFIX_LEN = len(PREFIX)
AOI_LINES = {"5":list(range(10,17)),"24":list(range(9,12)),"32":[3,8,16]+list(range(25,31))}

In [3]:
if not os.path.exists(os.path.join(BASE_DIR,OUT_DIR)):
    os.mkdir(os.path.join(BASE_DIR,OUT_DIR))

In [4]:
parser = Parser()
dbm_data = parser.read_all_data(DATABASE_MATRIX)
names,expertise = parser.get_expertise_CG(dbm_data,42)

In [5]:
exp = np.array(expertise)
np.sum(exp==0), np.sum(exp==1), np.sum(exp==2)

(8, 28, 18)

In [6]:
def get_fixation_stats(data,code_num):
    fixation_count = 0
    total_fixation_count = 0
    aoi_lines = AOI_LINES[code_num]
    for line in data:
        if line[0] == 0:
            continue
        total_fixation_count += line[1]
        if line[0] in aoi_lines:
            fixation_count += line[1]
            
    return total_fixation_count, fixation_count

In [7]:
for code in os.listdir(os.path.join(BASE_DIR,CODE_TYPE)):
    if not os.path.exists(os.path.join(BASE_DIR,OUT_DIR,CODE_TYPE)):
        os.mkdir(os.path.join(BASE_DIR,OUT_DIR,CODE_TYPE))
    print("CODE - ",code)
    subjects_list = []
    total_fixation_counts = []
    fixation_counts_aoi = []
    fixation_ratios_aoi = []
    expertises = []
    for i, file in enumerate(os.listdir(os.path.join(BASE_DIR,CODE_TYPE,code))):
        subjects_list.append("S"+str(i+1))
        if not os.path.exists(os.path.join(BASE_DIR,OUT_DIR,CODE_TYPE,code)):
            os.mkdir(os.path.join(BASE_DIR,OUT_DIR,CODE_TYPE,code))
        subject_name = file[PREFIX_LEN:-4-len(code)]
        data = pd.read_csv(os.path.join(BASE_DIR,CODE_TYPE,code,file))
        data = np.array(data)
        data = data[:,[0,-2,-1]]
        total_fixation_count, fixation_count = get_fixation_stats(data,code)
        print(subject_name,total_fixation_count, fixation_count)
        total_fixation_counts.append(total_fixation_count)
        fixation_counts_aoi.append(fixation_count)
        fixation_ratios_aoi.append(fixation_count/total_fixation_count)
        name_match = process.extractOne(subject_name.lower(),names)
        name_match = name_match[0]
        for idx, name in enumerate(names):
            if name.lower() == name_match.lower():
                expertises.append(expertise[idx])
                break
        
        #TODO: Plot fixation rate density
    plt.figure(figsize=(15,10))
    plt.bar(subjects_list,total_fixation_counts)
    plt.savefig(os.path.join(BASE_DIR,OUT_DIR,CODE_TYPE,code,"subjectVStotal_fixation.png"))
    plt.figure(figsize=(15,10))
    plt.bar(subjects_list,fixation_counts_aoi)
    plt.savefig(os.path.join(BASE_DIR,OUT_DIR,CODE_TYPE,code,"subjectVSfixation_counts_aoi.png"))
    plt.figure(figsize=(15,10))
    plt.bar(subjects_list,fixation_ratios_aoi)
    plt.savefig(os.path.join(BASE_DIR,OUT_DIR,CODE_TYPE,code,"subjectVSfixation_ratios_aoi.png"))
    plt.figure(figsize=(15,10))
    plt.scatter(expertises,fixation_ratios_aoi)
    plt.xticks([0,1,2],["expert","intermediate","novice"])
    plt.savefig(os.path.join(BASE_DIR,OUT_DIR,CODE_TYPE,code,"expertiseVSfixation_rate.png"))
    
    print("=================================================================================")

CODE -  24
nikesh 349 147
reddy 624 361
arpit 185 71
anuraag 554 265
knikhil 676 324
atrayee 242 40
rajdeep 648 374
niloy 173 121
arka 477 125
udhir 191 43
CODE -  5
meet 38 7
harshit 20 9
nikhil 199 52
farazul 124 29
rony 7 0
anshul 164 60
nitin 654 195
koushik 18 3
archit 392 118
CODE -  32
mayur 324 81
achal 374 90
diya 145 33
anjali 158 41
vamsi 237 41
aditya 473 121
arihant 590 192


In [16]:

for code in os.listdir(os.path.join(BASE_DIR,CODE_TYPE)):
    if not os.path.exists(os.path.join(BASE_DIR,OUT_DIR,CODE_TYPE)):
        os.mkdir(os.path.join(BASE_DIR,OUT_DIR,CODE_TYPE))
    subjects_list = []
    total_fixation_counts = []
    fixation_counts_aoi = []
    fixation_ratios_aoi = []
    for i, file in enumerate(os.listdir(os.path.join(BASE_DIR,CODE_TYPE,code))):
        subject_name = file[PREFIX_LEN:-4-len(code)]
        best_match = process.extractOne(subject_name.lower(),names)
        #print(subject_name.lower(),best_match)
        #mt = get_close_matches(subject_name.lower(),names,1)
        #if len(mt) != 0:
        #    print("\""+subject_name.lower()+"\" : \""+mt[0]+"\", ")